## Methods with parametric types

In [1]:
function adder{T}(a::T, b::T) T<:Number
    # add `a` and `b` together, where `a` and `b` are of type `T` that is a subset of `Number`
    a + b
end

adder (generic function with 1 method)

In [2]:
adder(1,2)  # Ints

3

In [3]:
adder(1., 2.)  # Floats

3.0

In [4]:
adder(1//2, 3//4)  # Rationals

5//4

In [5]:
adder(1, 2.)  # `a` and `b` are not the same type

LoadError: [91mMethodError: no method matching adder(::Int64, ::Float64)[0m
Closest candidates are:
  adder(::T, [91m::T[39m) where T at In[1]:1[39m

## Multiple dispatch

(Functions can have different methods that get dispatched according to the arguments of the function call)

In [6]:
function adder{T}(a::Vector{T}, b::T) T<:Number
    a .+ b  # element-wise operations use the `.` operator
end

adder (generic function with 2 methods)

In [7]:
adder([1,2,3], 1)

3-element Array{Int64,1}:
 2
 3
 4

In [8]:
adder([1.,2.,3.], 1.)

3-element Array{Float64,1}:
 2.0
 3.0
 4.0

In [9]:
adder([1,2,3], 1.)  # type mismatch

LoadError: [91mMethodError: no method matching adder(::Array{Int64,1}, ::Float64)[0m
Closest candidates are:
  adder(::Array{T,1}, [91m::T[39m) where T at In[6]:1
  adder(::T, [91m::T[39m) where T at In[1]:1[39m

# Adding more methods to adder

In [10]:
function adder{T}(a::Array{T}, b::Array{T, 2}) T<:Number  # element-wise addition of a column vector to a matrix
    if ndims(a) == 1
        a .+ b
    else
        a' .+ b  # transpose the row vector first
    end
end

adder (generic function with 3 methods)

In [11]:
adder([1,2], [1 2 3; 4 5 6])

2×3 Array{Int64,2}:
 2  3  4
 6  7  8

In [12]:
adder([1 2], [1 2 3; 4 5 6])

2×3 Array{Int64,2}:
 2  3  4
 6  7  8

In [13]:
adder([1;2], [1 2 3; 4 5 6])

2×3 Array{Int64,2}:
 2  3  4
 6  7  8

# A silly example of multiple dispatch

In [14]:
function adder{T}(a::Array{T}, b::Real) T<:Number
    c = Array{String}(size(a))
    for (idx, value) in enumerate(a)
        c[idx] = "$(value) + 2 is $(value + b)"
    end
    return c
end

adder (generic function with 4 methods)

In [15]:
adder(Vector(1:5), Int8(2))  # dispatch on the most specific method for adder(a,b)

5-element Array{String,1}:
 "1 + 2 is 3"
 "2 + 2 is 4"
 "3 + 2 is 5"
 "4 + 2 is 6"
 "5 + 2 is 7"

In [16]:
adder(Vector(1:5), 2//3)

5-element Array{String,1}:
 "1 + 2 is 5//3" 
 "2 + 2 is 8//3" 
 "3 + 2 is 11//3"
 "4 + 2 is 14//3"
 "5 + 2 is 17//3"

# User-defined types with multiple dispatch

In [17]:
struct MyType
    data::Number
end

In [18]:
x = MyType(1)
y = MyType(2)

MyType(2)

In [19]:
adder(x, y)  # doesn't work because Julia doesn't know what MyType + MyType means

LoadError: [91mMethodError: no method matching +(::MyType, ::MyType)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424[39m

In [20]:
import Base.+
+(a::MyType, b::MyType) = a.data + b.data  # so we have to define it

+ (generic function with 181 methods)

In [21]:
x + y

3

In [22]:
adder(x, y)  # now it works

3

# Type inheritence
(NB there are no classes in Julia)

In [23]:
abstract type MyNum end

In [24]:
+(a::MyNum, b::MyNum) = a.data + b.data  # define the operator for the abstract type

+ (generic function with 182 methods)

In [25]:
struct MyInt <: MyNum
    data::Int
end

struct MyFloat <: MyNum
    data::Float64
end

In [26]:
MyInt(1) + MyInt(2)

3

In [37]:
MyFloat(1.) + MyFloat(2.)

3.0

# Type stability is important for performance

In [28]:
using BenchmarkTools

In [29]:
function running_total(n)
    total = 0.
    for i=1:n
        total += i
    end
    return total
end

running_total (generic function with 1 method)

In [30]:
@benchmark running_total(1_000_000)  # slow

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.293 ms (0.00% GC)
  median time:      1.362 ms (0.00% GC)
  mean time:        1.397 ms (0.00% GC)
  maximum time:     4.024 ms (0.00% GC)
  --------------
  samples:          3571
  evals/sample:     1

Having to convert `i` to `Float` before adding to `total` by `promote(0., 1)`

In [31]:
@code_lowered 0. + 1

CodeInfo(:(begin 
        nothing
        return (Core._apply)(Base.+, (Base.promote)(x, y))
    end))

In [32]:
typeof(promote(1., 1))

Tuple{Float64,Float64}

In [33]:
function running_total_correct(n)
    total = 0  # initialise as an Int
    for i=1:n
        total += i
    end
    return total
end

running_total_correct (generic function with 1 method)

In [34]:
@benchmark running_total_correct(1_000_000)  #  ᕕ( ᐛ )ᕗ

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.271 ns (0.00% GC)
  median time:      3.285 ns (0.00% GC)
  mean time:        3.452 ns (0.00% GC)
  maximum time:     21.661 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000